In [7]:
# Part 1
#For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

#Start by creating a new Notebook for this assignment.
#Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

# I pulled up the web page, highlighted the table, cut & pasted to Excel and saved to a csv file, "Toronto.csv".
# I then imported the file into IBM Cloud Storage and used "Insert to code" function to create the following code:

# Start of "Insert to code segment"
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e4c63f1d09c549968caf743ebebfff9a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='d73P9G0wt07N6duy6tsqsYmwESOQl5apjZcKB3bEct-2',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_e4c63f1d09c549968caf743ebebfff9a.get_object(Bucket='segmentingandclusteringneighborho-donotdelete-pr-tbbplt7t0dhxdc',Key='Toronto.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
# End of "Insert to code segment"
    
df = pd.read_csv(body)
print("Original Shape", df.shape)

#Set width so output isn't split
pd.set_option('display.width', 150)
df.head()


Original Shape (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
is_Notassigned =  df['Neighbourhood']=='Not assigned'
dfNa = df[is_Notassigned]
dfNa_count = dfNa.count
if dfNa.empty:
    print("0 remaining rows have Neighborhood = Not assigned\n")
else:
    print(dfNa_count, "remaining rows have Nieghborhood = Not assigned\n")
    
#There are no rows left where Neighbourhood is "Not assigned"

#More than one neighborhood can exist in one postal code area. 
#Combine rows with the same postal code area into one row with the neighborhoods separated with a comma.
#The Wikipedia page presented no such rows, looks like they had already been combined!
#Just for yucks, I will check for duplicate rows based on the postal area code.
dfDup = df[df.duplicated(['Postal Code'])]
if dfDup.empty:
    print("There are 0 duplicate rows in the dataframe\n")
else:
    print("There are", dfDup.count, "duplicate rows in the dataframe\n")
print("Filtered Shape", df.shape, "\n")

df = df.reset_index()
del df['index']
df.head()

0 remaining rows have Neighborhood = Not assigned

There are 0 duplicate rows in the dataframe

Filtered Shape (103, 3) 



,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
# Part 2
#Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, 
#in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.
import pandas as pd
dfCoord = pd.read_csv('http://cocl.us/Geospatial_data')
print(dfCoord.head())
df2 = pd.merge(df, dfCoord, on='Postal Code')
print(df2.head())
print(df2.shape)

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
  Postal Code           Borough                                Neighbourhood   Latitude  Longitude
0         M3A        North York                                    Parkwoods  43.753259 -79.329656
1         M4A        North York                             Victoria Village  43.725882 -79.315572
2         M5A  Downtown Toronto                    Regent Park, Harbourfront  43.654260 -79.360636
3         M6A        North York             Lawrence Manor, Lawrence Heights  43.718518 -79.464763
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government  43.662301 -79.389494
(103, 5)
